**Paso 0: Librerias**

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

import joblib

**Paso 1: Limpieza avanzada y estandarización de variables**

In [2]:
df = pd.read_csv(r"C:\Users\javid\OneDrive\Escritorio\Javidev\ProjectML_Prediccion_Salarios_IT\data\survey_results_public.csv", low_memory=False)

# Renombramos las columnas más importantes
df = df.rename(columns={
    "Employment": "TipoEmpleo",
    "RemoteWork": "TrabajoRemoto",
    "DevType": "Rol",
    "EdLevel": "NivelEducativo",
    "YearsCodePro": "AniosExperiencia",
    "Country": "Pais",
    "OrgSize": "TamanoEmpresa",
    "ConvertedCompYearly": "SalarioAnual"
})

# Filtramos columnas relevantes
columnas_utiles = [
    "TipoEmpleo", "TrabajoRemoto", "Rol", "NivelEducativo",
    "AniosExperiencia", "Pais", "TamanoEmpresa", "SalarioAnual"
]
df = df[columnas_utiles]

# Limpiamos y transformamos la experiencia profesional
df["AniosExperiencia"] = df["AniosExperiencia"].replace({
    "Less than 1 year": "0",
    "More than 50 years": "51"
})
df["AniosExperiencia"] = pd.to_numeric(df["AniosExperiencia"], errors="coerce")

# Agrupamos países poco frecuentes como "Otro"
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")

# Convertimos tipo_empleo en categorías resumidas
def simplificar_tipo_empleo(valor):
    if pd.isna(valor): return None
    if "full-time" in valor: return "JornadaCompleta"
    if "part-time" in valor: return "MediaJornada"
    if "freelancer" in valor or "contractor" in valor: return "Autonomo"
    if "student" in valor: return "Estudiante"
    if "not employed" in valor: return "Desempleado"
    if "retired" in valor: return "Jubilado"
    return "Otro"

df["TipoEmpleo"] = df["TipoEmpleo"].apply(simplificar_tipo_empleo)

# Simplificamos modalidad de trabajo
df["TrabajoRemoto"] = df["TrabajoRemoto"].replace({
    "Remote": "Remoto",
    "In-person": "Presencial",
    "Hybrid (some remote, some in-person)": "Hibrido"
})

def mapear_rol(rol_raw):
    mapa_roles = {
        "Developer, full-stack": "Developer_Full_Stack",
        "Developer, back-end": "Developer_Back_End",
        "Developer, front-end": "Developer_Front_End",
        "Developer, desktop or enterprise applications": "Developer_Desktop_Enterprise",
        "Developer, mobile": "Developer_Mobile",
        "Developer, embedded applications or devices": "Developer_Embedded_Devices",
        "Data engineer": "Data_Engineer",
        "Engineering manager": "Engineering_Manager",
        "DevOps specialist": "DevOps_Specialist",
        "Data scientist or machine learning specialist": "Data_Scientist_ML",
        "Research & Development role": "Investigación_Desarrollo",
        "Academic researcher": "Investigador_Academico",
        "Cloud infrastructure engineer": "Cloud_Infrastructure_Engineer",
        "Senior Executive (C-Suite, VP, etc.)": "Senior_Executive",
    }

    if pd.isna(rol_raw):
        return "Otro_rol"

    # Algunos registros tienen varios roles separados por ';', nos quedamos con el primero
    primer_rol = rol_raw.split(";")[0].strip()

    # Mapear el primer rol, o poner 'Otro_rol' si no está en el diccionario
    return mapa_roles.get(primer_rol, "Otro_rol")

# Aplicarlo
df["Rol"] = df["Rol"].apply(mapear_rol)

# Simplificamos nivel educativo
def simplificar_nivel(nivel):
    if pd.isna(nivel): return None
    if "Bachelor" in nivel: return "Grado_universitario"
    if "Master" in nivel: return "Master"
    if "Professional" in nivel or "Ph.D" in nivel or "Doctoral" in nivel: return "Doctorado"
    if "Secondary" in nivel: return "Secundaria"
    if "Primary" in nivel: return "Primaria"
    if "Associate" in nivel: return "Grado_medio"
    if "Some college" in nivel: return "Universidad_sin_titulo"
    return "Otro_nivel_educativo"

df["NivelEducativo"] = df["NivelEducativo"].apply(simplificar_nivel)

# Eliminamos filas nulas y salarios excesivos
# df = df.dropna()
df = df[df["SalarioAnual"] <= 300000]


**Paso 2: Procesamiento de la columna 'Rol' y 'Pais' y 'Tamano_empresa'**

In [3]:
df["AniosExperiencia"] = df["AniosExperiencia"].replace({
    "Less than 1 year": "0", "More than 50 years": "51"
})
df["AniosExperiencia"] = pd.to_numeric(df["AniosExperiencia"], errors="coerce")
df["TamanoEmpresa"] = df["TamanoEmpresa"].replace("I don’t know", pd.NA)

In [4]:
df = df.dropna()
df = df[df["SalarioAnual"] <= 300000]

**Paso 2: Procesamiento de la columna 'Rol' y 'Pais' y 'Tamano_empresa'**

In [5]:
# Rol
roles_dummies = df["Rol"].str.get_dummies(sep=";")
top_roles = roles_dummies.sum().sort_values(ascending=False).head(15).index
df = pd.concat([df.drop(columns="Rol"), roles_dummies[top_roles]], axis=1)

# País
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro_pais")

# Aquí el cambio:
df = pd.get_dummies(df, columns=["Pais", "TipoEmpleo", "TrabajoRemoto", "NivelEducativo"], prefix='', prefix_sep='')

# Y como ya no hay columna "Pais_Otro", el drop cambia a simplemente:
if "Otro_pais" in df.columns:
    df = df.drop(columns=["Otro_pais"])

def traducir_tamano_empresa(x):
    traducciones = {
        "Just me - I am a freelancer, sole proprietor, etc.": "Freelance",
        "2 to 9 employees": "2-9_empleados",
        "10 to 19 employees": "10-19_empleados",
        "20 to 99 employees": "20-99_empleados",
        "100 to 499 employees": "100-499_empleados",
        "500 to 999 employees": "500-999_empleados",
        "1,000 to 4,999 employees": "1000-4999_empleados",
        "5,000 to 9,999 employees": "5000-9999_empleados",
        "10,000 or more employees": "+10000_empleados"
    }
    return traducciones.get(x, x)  # si no lo encuentra, deja el valor original

# Aplicarlo al dataframe
df["TamanoEmpresa"] = df["TamanoEmpresa"].apply(traducir_tamano_empresa)


**Paso 3: Codificación de columnas categóricas**

In [6]:
for col in df.select_dtypes(include="bool").columns:
    df[col] = df[col].astype(int)

**Paso 4: Limpieza y preparación final**

In [7]:
# # 2. Limpiar textos
# def limpiar_texto(x):
#     if isinstance(x, str):
#         x = x.strip()
#         x = x.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").replace("ñ", "n")
#         return x
#     return x

# for columna in ["TipoEmpleo", "TrabajoRemoto", "NivelEducativo", "Pais"]:
#     if columna in df.columns:
#         df[columna] = df[columna].apply(limpiar_texto)

# if "Rol" in df.columns:
#     df["Rol"] = df["Rol"].apply(lambda roles: [limpiar_texto(r) for r in roles] if isinstance(roles, list) else [])

# Preparar datos finales
X = df.drop(columns="SalarioAnual")
X.columns = [re.sub(r"[^a-zA-Z0-9]", "_", col) for col in X.columns]
y = df["SalarioAnual"]


**Paso 5: División en train/test**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

**Paso 6: Definición del grid de hiperparámetros**

In [9]:
param_grid = {
    "iterations": [800],
    "learning_rate": [0.03],
    "depth": [6],
    "l2_leaf_reg": [3],
    "bagging_temperature": [0.2]
}

**Paso 7: GridSearchCV y entrenamiento de CatBoost**

In [10]:
model = CatBoostRegressor(verbose=0, random_state=42)

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="r2",
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train, cat_features=["TamanoEmpresa"])

print("Mejor configuración encontrada:")
print(grid.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Mejor configuración encontrada:
{'bagging_temperature': 0.2, 'depth': 6, 'iterations': 800, 'l2_leaf_reg': 3, 'learning_rate': 0.03}


**Paso 8: Evaluación contra test**

In [11]:
# Evaluación en test
mejor_modelo = grid.best_estimator_
y_pred = mejor_modelo.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"R² en test con mejores hiperparámetros: {r2:.4f}")


R² en test con mejores hiperparámetros: 0.5917


**Paso 9: Guardado de modelo**

In [12]:
columnas_modelo = X.columns.tolist()
joblib.dump((mejor_modelo, columnas_modelo), "model.pkl")

['model.pkl']

In [13]:
list(X.head().columns)

['AniosExperiencia',
 'TamanoEmpresa',
 'Developer_Full_Stack',
 'Developer_Back_End',
 'Otro_rol',
 'Developer_Front_End',
 'Developer_Desktop_Enterprise',
 'Developer_Mobile',
 'Developer_Embedded_Devices',
 'Data_Engineer',
 'Engineering_Manager',
 'DevOps_Specialist',
 'Data_Scientist_ML',
 'Investigaci_n_Desarrollo',
 'Investigador_Academico',
 'Cloud_Infrastructure_Engineer',
 'Senior_Executive',
 'Australia',
 'Brazil',
 'Canada',
 'France',
 'Germany',
 'India',
 'Italy',
 'Netherlands',
 'Otro',
 'Poland',
 'Spain',
 'Sweden',
 'Ukraine',
 'United_Kingdom_of_Great_Britain_and_Northern_Ireland',
 'United_States_of_America',
 'Autonomo',
 'JornadaCompleta',
 'MediaJornada',
 'Hibrido',
 'Presencial',
 'Remoto',
 'Doctorado',
 'Grado_medio',
 'Grado_universitario',
 'Master',
 'Otro_nivel_educativo',
 'Primaria',
 'Secundaria',
 'Universidad_sin_titulo']